In [1]:
# Parser Import
from data_source.parseData.googleFitDataParsing import *

In [5]:
processor = ParseData()
googleFit_allData = processor.allData_json(r'C:\Users\DELL\Downloads\Google Fit Data\Hamdaan Fit Data\takeout-20241007T045109Z-002\Takeout\Fit\All Data')
googleFit_activitiesData = processor.activities_tcx(r'C:\Users\DELL\Downloads\Google Fit Data\Hamdaan Fit Data\takeout-20241007T045109Z-002\Takeout\Fit\Activities')
user_name = "5cbdc5c5-2793-452b-a95d-0dbd3a8719ce"

start_date = '2024-10-04'
days_offset = 7
offset_sign = '-'
date_list = ['2024-09-13', '2024-09-09', '2024-09-07']
end_date = '2024-10-01'

# Add the new column to both DataFrames
googleFit_allData['userName'] = user_name
googleFit_activitiesData['userName'] = user_name

# Reorder the columns to position 'userName' as the first column
allData_columns = ['userName'] + [col for col in googleFit_allData.columns if col != 'userName']
activitiesData_columns = ['userName'] + [col for col in googleFit_activitiesData.columns if col != 'userName']

googleFit_allData = googleFit_allData[allData_columns]
googleFit_activitiesData = googleFit_activitiesData[activitiesData_columns]

In [61]:
import pandas as pd
from datetime import datetime, timedelta

class WDuration:

    def __init__(self, workouts_df, *args):
        self.workouts_df = workouts_df
        
        # Convert columns to datetime
        self.workouts_df["Lap.StartTime"] = pd.to_datetime(self.workouts_df["Lap.StartTime"]).dt.tz_localize(None)
        self.workouts_df["Lap.Track.Trackpoint.Time"] = pd.to_datetime(self.workouts_df["Lap.Track.Trackpoint.Time"]).dt.tz_localize(None)
        
        # Calculate duration based on Lap.Track.Trackpoint.Time
        self.workouts_df["Duration (Minutes)"] = self.workouts_df.groupby("Lap.StartTime")["Lap.Track.Trackpoint.Time"].transform(lambda x: (x.max() - x.min()).total_seconds() / 60)
        self.workouts_df["Duration (Minutes)"] = self.workouts_df["Duration (Minutes)"].round(3)

        # Filter data based on arguments
        if len(args) == 1 and isinstance(args[0], list):
            self.dates_list = [pd.to_datetime(date).tz_localize(None) for date in args[0]]
            self.filtered_workouts_df = self._filter_by_dates_list(self.workouts_df, self.dates_list)
        elif len(args) == 1:
            self.start_date = pd.to_datetime(args[0]).tz_localize(None)
            self.filtered_workouts_df = self._filter_by_single_date(self.workouts_df, self.start_date)
        elif len(args) == 2:
            self.start_date = pd.to_datetime(args[0]).tz_localize(None)
            self.end_date = pd.to_datetime(args[1]).tz_localize(None)
            self.filtered_workouts_df = self._filter_by_date_range(self.workouts_df, self.start_date, self.end_date)
        elif len(args) == 3:
            self.start_date = pd.to_datetime(args[0]).tz_localize(None)
            self.days_offset = int(args[1])
            self.offset_sign = args[2]
            self.filtered_workouts_df = self._filter_by_offset(self.workouts_df, self.start_date, self.days_offset, self.offset_sign)

        if self.filtered_workouts_df.empty:
            self._handle_empty_records()

    def _filter_by_single_date(self, df, start_date):
        return self._filter_data(df, start_date)

    def _filter_by_date_range(self, df, start_date, end_date):
        return self._filter_data(df, start_date, end_date)

    def _filter_by_offset(self, df, start_date, days_offset, offset_sign):
        if offset_sign == '+':
            end_date = start_date + timedelta(days=days_offset)
        elif offset_sign == '-':
            end_date = start_date
            start_date = start_date - timedelta(days=days_offset)
        return self._filter_data(df, start_date, end_date)

    def _filter_by_dates_list(self, df, dates_list):
        filtered_df = pd.concat([self._filter_data(df, date) for date in dates_list])
        return filtered_df.drop_duplicates().reset_index(drop=True)

    def _filter_data(self, df, start_date, end_date=None):
        if df.empty:
            return pd.DataFrame(columns=['userName', 'Sport', 'Lap.StartTime', 'Lap.Track.Trackpoint.Time', 'Duration (Minutes)'])
        
        start_of_day = start_date.replace(hour=0, minute=0, second=0, microsecond=0)
        end_of_day = end_date.replace(hour=23, minute=59, second=59, microsecond=999999) if end_date else start_of_day.replace(hour=23, minute=59, second=59, microsecond=999999)

        df["Id"] = pd.to_datetime(df["Id"]).dt.tz_localize(None)
        filtered_df = df[(df['Id'] >= start_of_day) & (df['Id'] <= end_of_day)]
        print(filtered_df)
        
        return filtered_df.reset_index(drop=True)

    def _handle_empty_records(self):
        print("No workout data available for the specified dates.")

    def process(self):
        if self.filtered_workouts_df.empty:
            print("No data available for processing.")
            return pd.DataFrame(columns=[
                'userName', 'valueGeneratedAt', 'Sport', 'Lap.StartTime', 'Lap.Track.Trackpoint.Time', 'Duration (Minutes)', 'unit', 'value'
            ])

        self.filtered_workouts_df['valueGeneratedAt'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        self.filtered_workouts_df['unit'] = 'min'

        final_workout_df = self.filtered_workouts_df.rename(columns={'Duration (Minutes)': 'value'})

        # Reorder columns to include Sport before Lap.StartTime
        final_workout_df = final_workout_df[['userName', 'valueGeneratedAt', 'Sport', 'Lap.StartTime', 'Lap.Track.Trackpoint.Time', 'unit', 'value']]
        final_workout_df['value'] = final_workout_df['value'].round(1)
        #final_workout_df = final_workout_df.drop_duplicates(subset=["Sport", "Lap.StartTime"], keep="first").reset_index(drop=True)

        return final_workout_df


In [63]:
df = WDuration(googleFit_activitiesData, '2024-10-04').process()
df

                                 userName              Sport         Id  \
193  5cbdc5c5-2793-452b-a95d-0dbd3a8719ce  Strength_training 2024-10-04   
194  5cbdc5c5-2793-452b-a95d-0dbd3a8719ce  Strength_training 2024-10-04   
195  5cbdc5c5-2793-452b-a95d-0dbd3a8719ce  Strength_training 2024-10-04   
196  5cbdc5c5-2793-452b-a95d-0dbd3a8719ce  Strength_training 2024-10-04   
197  5cbdc5c5-2793-452b-a95d-0dbd3a8719ce  Strength_training 2024-10-04   
198  5cbdc5c5-2793-452b-a95d-0dbd3a8719ce  Strength_training 2024-10-04   
199  5cbdc5c5-2793-452b-a95d-0dbd3a8719ce  Strength_training 2024-10-04   
200  5cbdc5c5-2793-452b-a95d-0dbd3a8719ce  Strength_training 2024-10-04   
201  5cbdc5c5-2793-452b-a95d-0dbd3a8719ce  Strength_training 2024-10-04   
202  5cbdc5c5-2793-452b-a95d-0dbd3a8719ce  Strength_training 2024-10-04   
203  5cbdc5c5-2793-452b-a95d-0dbd3a8719ce  Strength_training 2024-10-04   
204  5cbdc5c5-2793-452b-a95d-0dbd3a8719ce            Walking 2024-10-04   
205  5cbdc5c5-2793-452b-a

,userName,valueGeneratedAt,Sport,Lap.StartTime,Lap.Track.Trackpoint.Time,unit,value
0,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-10-10 17:56:32,Strength_training,2024-10-04 03:18:20,2024-10-04 03:18:20,min,45.4
1,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-10-10 17:56:32,Strength_training,2024-10-04 03:18:20,2024-10-04 03:20:00,min,45.4
2,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-10-10 17:56:32,Strength_training,2024-10-04 03:18:20,2024-10-04 03:25:00,min,45.4
3,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-10-10 17:56:32,Strength_training,2024-10-04 03:18:20,2024-10-04 03:30:00,min,45.4
4,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-10-10 17:56:32,Strength_training,2024-10-04 03:18:20,2024-10-04 03:35:00,min,45.4
5,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-10-10 17:56:32,Strength_training,2024-10-04 03:18:20,2024-10-04 03:40:00,min,45.4
6,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-10-10 17:56:32,Strength_training,2024-10-04 03:18:20,2024-10-04 03:45:00,min,45.4
7,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-10-10 17:56:32,Strength_training,2024-10-04 03:18:20,2024-10-04 03:50:00,min,45.4
8,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-10-10 17:56:32,Strength_training,2024-10-04 03:18:20,2024-10-04 03:55:00,min,45.4
9,5cbdc5c5-2793-452b-a95d-0dbd3a8719ce,2024-10-10 17:56:32,Strength_training,2024-10-04 03:18:20,2024-10-04 04:00:00,min,45.4
